In [ ]:
# Install and import packages
!pip install pandas openpyxl matplotlib plotly -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from google.colab import files
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
print("Setup complete!")

Setup complete!


In [ ]:
# Upload your categorized Excel file
print("Please upload: categorized_orders_clean.xlsx")
uploaded = files.upload()
filename = list(uploaded.keys())[0]
print(f"\nUploaded: {filename}")

Please upload: categorized_orders_clean.xlsx


Saving categorized_orders_clean.xlsx to categorized_orders_clean.xlsx

Uploaded: categorized_orders_clean.xlsx


In [ ]:
# Load data
df = pd.read_excel(filename)
print(f"Loaded {len(df)} rows")
print(f"Columns: {df.columns.tolist()}")

# Get order column (first column)
order_col = df.columns[0]

# Filter to parent rows only (rows with order number)
parent_df = df[df[order_col].notna()].copy()
print(f"\n{len(parent_df)} unique orders")

Loaded 2482 rows
Columns: ['Order No', 'Outlet', 'Order Type', 'Order Date', 'Contact no', 'Amount', 'Reason', 'Remark', 'Void By ', 'Void Date', 'Placed By', 'Order Time', 'Item', 'Qty', 'Rate', 'Predicted_Category', 'Extracted_New_Bill']

713 unique orders


In [ ]:
# Get month from data for report title
if 'Order Date' in parent_df.columns:
    try:
        dates = pd.to_datetime(parent_df['Order Date'], errors='coerce')
        if dates.notna().any():
            month_str = dates.dropna().iloc[0].strftime('%B, %Y')
        else:
            month_str = "Current Period"
    except:
        month_str = "Current Period"
else:
    month_str = "Current Period"

display(HTML(f'''
<div style="text-align:center; padding:20px; color:white; border-radius:10px;">
    <h1>VOID BILLS ANALYSIS</h1>
    <h3>Month of {month_str}</h3>
</div>
'''))

---
## 1. Findings
### 1.1 Identification of Special Incidents

In [ ]:
# Key statistics
total_voids = len(parent_df)

# Top outlet
outlet_counts = parent_df['Outlet'].value_counts()
top_outlet = outlet_counts.index[0]
top_outlet_count = outlet_counts.iloc[0]

# Main reason
reason_counts = parent_df['Predicted_Category'].value_counts()
main_reason = reason_counts.index[0]
main_reason_count = reason_counts.iloc[0]

# Order types
order_types = parent_df['Order Type'].dropna().unique()[:4]
order_types_str = ", ".join(order_types)

display(HTML(f'''
<div style="padding:15px; border-left:4px solid #1976D2; margin:10px 0;">
    <h4>Key Findings:</h4>
    <ol>
        <li>The total number of void ({order_types_str}) bills was <b>{total_voids}</b>.</li>
        <li>The <b>{top_outlet}</b> outlet had the highest void bills. It was <b>{top_outlet_count}</b>.</li>
        <li>The main reason for the highest number of bill voided was <b>{main_reason}</b>. It was <b>{main_reason_count}</b>.</li>
    </ol>
</div>
'''))

### 1.2 Void Bills with Significant Total Amount

In [ ]:
# Top void bills by amount
cols = [order_col, 'Outlet', 'Order Type', 'Order Date', 'Reason', 'Void By ', 'Amount']
available_cols = [c for c in cols if c in parent_df.columns]

top_amount = parent_df.nlargest(10, 'Amount')[available_cols].copy()
top_amount['Amount'] = top_amount['Amount'].apply(lambda x: f"{x:,.2f}")

display(HTML("<h4>Top 10 Void Bills by Amount:</h4>"))
display(top_amount.style.set_table_styles([{'selector': 'th', 'props': [('background-color', '#1976D2'), ('color', 'white')]}]))

,Order No,Outlet,Order Type,Order Date,Reason,Void By,Amount
2354,MM6241,Trincomalee,Delivery,2026-01-24 00:00:00,cx want to change the time,34652.000000,"20,521.75"
148,N50066,Bambalapitiya,Dine In,2026-01-14 00:00:00,Customer wanted to change the order,6391.000000,"17,974.00"
2409,U54642,Union Place,Take Away,2026-01-27 00:00:00,Customer wanted to change the order,31988.000000,"16,745.03"
776,MO3899,Kadawatha,Delivery,2026-01-27 00:00:00,Cux don't Have Money for settle this order,23266.000000,"15,641.25"
1367,U52727,Kurunegala 2,Take Away,2026-01-10 00:00:00,Customer wanted to change the order,29737.000000,"15,450.03"
2378,U50388,Union Place,Dine In,2026-01-04 00:00:00,Customer wanted to change the order,29082.000000,"15,447.75"
256,A47373,Colombo City Center PH,Aggregator,2026-01-01 00:00:00,Due to Out Of Stock,28748.000000,"15,174.83"
1382,U53207,Kurunegala 2,Take Away,2026-01-15 00:00:00,Customer wanted to change the order,29737.000000,"14,525.02"
425,U68934,Dehiwela,Dine In,2026-01-23 00:00:00,Customer want to have a Promotion,11536.000000,"14,469.50"
1924,N72513,Nuwara Eliya,Dine In,2026-01-23 00:00:00,Customer denied the order,9506.000000,"13,899.75"


---
## 2. Facts
### 2.1 Channel-wise Number of Void Bills

In [ ]:
# Channel-wise pivot table
channel_pivot = pd.crosstab(parent_df['Outlet'], parent_df['Order Type'], margins=True, margins_name='Total')
channel_pivot = channel_pivot.sort_values('Total', ascending=False)

display(HTML("<h4>Channel-wise Void Bills by Outlet:</h4>"))
display(channel_pivot.head(20).style.background_gradient(cmap='Blues').set_table_styles([{'selector': 'th', 'props': [('background-color', '#1976D2'), ('color', 'white')]}]))

Order Type,Aggregator,Delivery,Dine In,Take Away,Total
Outlet,,,,,
Total,51,263,153,246,713
Dehiwela,0,12,8,5,25
Negambo,9,9,2,4,24
Trincomalee,0,4,6,12,22
Kochchikade,17,2,1,2,22
Rajagiriya,2,11,3,5,21
Katugasthota,10,3,6,1,20
Colombo City Center PH,5,0,9,2,16
Dambulla,0,1,7,7,15


In [ ]:
# Channel-wise bar chart
chart_data = channel_pivot.drop('Total', errors='ignore').drop('Total', axis=1, errors='ignore').head(15)

fig = go.Figure()
colors = {'Aggregator': '#4CAF50', 'Delivery': '#2196F3', 'Dine In': '#FFC107', 'Take Away': '#9C27B0'}

for col in chart_data.columns:
    fig.add_trace(go.Bar(name=col, x=chart_data.index, y=chart_data[col], marker_color=colors.get(col, '#607D8B')))

fig.update_layout(title='Channel-wise Void Bills', barmode='group', xaxis_tickangle=-45, height=500,
                  legend=dict(orientation="h", yanchor="bottom", y=1.02))
fig.show()

### 2.2 Outlets-wise Highest Total Void Bill Value

In [ ]:
# Value by outlet and order type
value_pivot = parent_df.pivot_table(values='Amount', index='Outlet', columns='Order Type', aggfunc='sum', fill_value=0)
value_pivot['Total (Rs.)'] = value_pivot.sum(axis=1)
value_pivot = value_pivot.sort_values('Total (Rs.)', ascending=False)

# Format for display
value_display = value_pivot.copy()
for col in value_display.columns:
    value_display[col] = value_display[col].apply(lambda x: f"{x:,.2f}")

display(HTML("<h4>Outlet-wise Total Void Bill Value:</h4>"))
display(value_display.head(15).style.set_table_styles([{'selector': 'th', 'props': [('background-color', '#1976D2'), ('color', 'white')]}]))

Order Type,Aggregator,Delivery,Dine In,Take Away,Total (Rs.)
Outlet,,,,,
Trincomalee,0.00,"32,400.50","32,233.34","46,435.07","111,068.91"
Dehiwela,0.00,"47,966.50","37,872.25","11,378.52","97,217.27"
Rajagiriya,"8,315.61","46,895.80","8,761.25","14,830.01","78,802.67"
Kotahena,0.00,"61,824.86","9,954.50",0.00,"71,779.36"
Katugasthota,"22,772.29","11,997.00","31,637.25","3,935.01","70,341.54"
Colombo City Center PH,"29,286.95",0.00,"34,345.18","5,808.01","69,440.13"
Dambulla,0.00,"3,418.50","38,087.25","24,076.53","65,582.28"
Kundasale,0.00,"21,782.73","5,332.00","33,000.04","60,114.77"
Nuwara Eliya,0.00,0.00,"30,809.50","28,670.04","59,479.54"


In [ ]:
# Pie chart
top_outlets = value_pivot['Total (Rs.)'].head(10)
fig = px.pie(values=top_outlets.values, names=top_outlets.index, title='TOTAL VOID BILL VALUE (Top 10 Outlets)', hole=0.3)
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.update_layout(height=500)
fig.show()

### 2.3 Summarize the Reasons for Void Bills

In [ ]:
# Top 5 reasons
top_2_reasons = reason_counts.head(5).reset_index()
top_2_reasons.columns = ['Void Reason', 'Number of Void']

display(HTML("<h4>Most Common Reasons:</h4>"))
display(top_2_reasons.style.set_table_styles([{'selector': 'th', 'props': [('background-color', '#1976D2'), ('color', 'white')]}]))

,Void Reason,Number of Void
0,cus. Change the order,154
1,promotion,103
2,Customer denied the order,83
3,Cashier mistake,65
4,phone,50


### 2.4 Top Void Reason Category #1

In [ ]:
# Top category #1 orders (dynamic)
top_category_1 = reason_counts.index[0]
top_1_orders = parent_df[parent_df['Predicted_Category'] == top_category_1].sort_values('Amount', ascending=False)

display_cols = [order_col, 'Outlet', 'Order Type', 'Order Date', 'Reason', 'Void By ', 'Amount']
avail_cols = [c for c in display_cols if c in top_1_orders.columns]

top_1_display = top_1_orders[avail_cols].head(20).copy()
if 'Amount' in top_1_display.columns:
    top_1_display['Amount'] = top_1_display['Amount'].apply(lambda x: f"{x:,.2f}")

# Get friendly name if available
top_1_name = name_map.get(top_category_1, top_category_1) if 'name_map' in dir() else top_category_1

display(HTML(f"<h4>Bills Voided for '{top_1_name}' ({len(top_1_orders)} orders):</h4>"))
display(top_1_display.style.set_table_styles([{'selector': 'th', 'props': [('background-color', '#1976D2'), ('color', 'white')]}]))

,Order No,Outlet,Order Type,Order Date,Reason,Void By,Amount
2354,MM6241,Trincomalee,Delivery,2026-01-24 00:00:00,cx want to change the time,34652.000000,"20,521.75"
1367,U52727,Kurunegala 2,Take Away,2026-01-10 00:00:00,Customer wanted to change the order,29737.000000,"15,450.03"
2378,U50388,Union Place,Dine In,2026-01-04 00:00:00,Customer wanted to change the order,29082.000000,"15,447.75"
2468,Y37699,Yakkala,Dine In,2026-01-04 00:00:00,Customer wanted to change the order,25810.000000,"13,577.25"
217,ML2723,Borella,Delivery,2026-01-22 00:00:00,CUSTOMER NEED TO BE 12.30 PM,23266.000000,"13,115.00"
810,K37055,Kandana,Dine In,2026-01-06 00:00:00,Customer wanted to change the order,24832.000000,"12,993.53"
59,Z27308,Arayampathy,Take Away,2026-01-10 00:00:00,Customer wanted to change the order,21649.000000,"12,345.02"
260,A47411,Colombo City Center PH,Dine In,2026-01-02 00:00:00,Customer wanted to change the order,36042.000000,"12,104.50"
1864,LZ6122,Nugegoda,Delivery,2026-01-03 00:00:00,time change,24970.000000,"9,954.50"
1994,MH4319,Pelawatta,Delivery,2026-01-15 00:00:00,order was changed to 89,34652.000000,"9,314.88"


### 2.5 Top Void Reason Category #2

In [ ]:
# Top category #2 orders (dynamic)
top_category_2 = reason_counts.index[1] if len(reason_counts) > 1 else reason_counts.index[0]
top_2_orders = parent_df[parent_df['Predicted_Category'] == top_category_2].sort_values('Amount', ascending=False)

top_2_display = top_2_orders[avail_cols].head(25).copy()
if 'Amount' in top_2_display.columns:
    top_2_display['Amount'] = top_2_display['Amount'].apply(lambda x: f"{x:,.2f}")

# Get friendly name if available
top_2_name = name_map.get(top_category_2, top_category_2) if 'name_map' in dir() else top_category_2

display(HTML(f"<h4>Bills Voided for '{top_2_name}' ({len(top_2_orders)} orders):</h4>"))
display(top_2_display.style.set_table_styles([{'selector': 'th', 'props': [('background-color', '#1976D2'), ('color', 'white')]}]))

,Order No,Outlet,Order Type,Order Date,Reason,Void By,Amount
1382,U53207,Kurunegala 2,Take Away,2026-01-15 00:00:00,Customer wanted to change the order,29737.000000,"14,525.02"
308,D73221,Dambulla,Dine In,2026-01-09 00:00:00,Customer wanted to change the order,22203.000000,"12,136.75"
1854,N77080,Nittambuwa,Dine In,2026-01-21 00:00:00,Customer want to have a Promotion,2169.000000,"11,226.12"
768,K45189,Kadawatha,Dine In,2026-01-10 00:00:00,Customer want to have a Promotion,30234.000000,"10,922.00"
2165,R47631,Ragama,Dine In,2026-01-15 00:00:00,Customer wanted to change the order,24716.000000,"10,750.00"
15,L41883,Aluthgama,Take Away,2026-01-10 00:00:00,Customer wanted to change the order,27519.000000,"10,630.02"
1299,U50070,Kundasale,Take Away,2026-01-21 00:00:00,Customer want to have a Promotion,22204.000000,"9,680.02"
881,MR0169,Katugasthota,Delivery,2026-01-30 00:00:00,"customer placed 02 offers for the order, outlet showed only one offer, therefore order no.150 placed",23266.000000,"8,772.00"
361,U66450,Dehiwela,Dine In,2026-01-02 00:00:00,Customer want to have a Promotion,23277.000000,"8,127.00"
643,H06525,Havelock,Take Away,2026-01-21 00:00:00,Customer want to have a Promotion,24228.000000,"8,040.01"


### 2.6 The Reasons for All Void Bills

In [ ]:
# All reasons breakdown
all_reasons = reason_counts.reset_index()
all_reasons.columns = ['Void Reason', 'Number Of Void']

# Friendly names mapping
name_map = {
    'cus. Change the order': 'Change of customer request',
    'promotion': 'Promotion',
    'Cashier mistake': 'Cashier mistake',
    'Customer denied the order': 'Customer denied the order',
    'Customer Cancel order': 'Customer Cancel order',
    'cus.related issue': 'Customer Related Issue',
    'grid issue': 'Grid issue',
    'phone': 'Contact Number Issues',
    'order without reason/ remark': 'Orders Without Reason / Remark',
    'Order delay': 'Order Delay',
    'order type change': 'Order Type Change',
    'system issue': 'System issue / breakdown',
    'rider issue': 'Riders related issues',
    'double punch': 'Order Double Punched',
    'payment issue': 'Payment Issues',
    'out of stock': 'Out of Stock',
    'location': 'Location',
    'testing': 'Testing',
    'Call Center mistake': 'CSR Issue',
    'product issue or complain': 'Product issue or complain',
    'order cancelled by aggregator': 'Order cancelled by aggregator',
    'other': 'Other'
}

all_reasons['Void Reason'] = all_reasons['Void Reason'].map(name_map).fillna(all_reasons['Void Reason'])

display(HTML("<h4>Complete Breakdown of Void Reasons:</h4>"))
display(all_reasons.style.set_properties(**{'background-color': ''}).set_table_styles([{'selector': 'th', 'props': [('background-color', '#FF8F00')]}]))

,Void Reason,Number Of Void
0,Change of customer request,154
1,Promotion,103
2,Customer denied the order,83
3,Cashier mistake,65
4,Contact Number Issues,50
5,voids without clear reason/ remark,50
6,Customer Cancel order,36
7,Location,23
8,Testing,23
9,Order Type Change,19


In [ ]:
# Bar chart for all reasons
fig = px.bar(all_reasons, x='Void Reason', y='Number Of Void', title='Void Bills by Reason Category',
             color='Number Of Void', color_continuous_scale='Blues')
fig.update_layout(xaxis_tickangle=-45, height=500, showlegend=False)
fig.show()

### 2.7 Delayed Void Processing Time

In [ ]:
# Time gap analysis
if 'Order Time' in parent_df.columns and 'Void Date' in parent_df.columns:
    delayed = parent_df.copy()
    try:
        delayed['Order_DT'] = pd.to_datetime(delayed['Order Time'], errors='coerce')
        delayed['Void_DT'] = pd.to_datetime(delayed['Void Date'], errors='coerce')
        delayed['Time_Gap_Seconds'] = (delayed['Void_DT'] - delayed['Order_DT']).dt.total_seconds()

        # Filter significant delays (> 2 hours = 7200 seconds)
        delayed_orders = delayed[delayed['Time_Gap_Seconds'] > 7200].sort_values('Time_Gap_Seconds', ascending=False)

        # Format time gap as HH:MM:SS
        def format_time_gap(seconds):
            if pd.isna(seconds):
                return ""
            hours = int(seconds // 3600)
            minutes = int((seconds % 3600) // 60)
            secs = int(seconds % 60)
            return f"{hours:02d}:{minutes:02d}:{secs:02d}"

        if len(delayed_orders) > 0:
            delay_cols = [order_col, 'Outlet', 'Reason', 'Order_DT', 'Void_DT', 'Time_Gap_Seconds']
            delay_display = delayed_orders[[c for c in delay_cols if c in delayed_orders.columns]].head(15).copy()
            delay_display['Time Gap'] = delay_display['Time_Gap_Seconds'].apply(format_time_gap)
            delay_display = delay_display.drop('Time_Gap_Seconds', axis=1)

            display(HTML(f"<h4>Orders with Delayed Void Processing ({len(delayed_orders)} orders >2hr):</h4>"))
            display(delay_display.style.set_table_styles([{'selector': 'th', 'props': [('background-color', '#1976D2'), ('color', 'white')]}]))
        else:
            print("No significantly delayed orders found.")
    except Exception as e:
        print(f"Could not calculate time gaps: {e}")
else:
    print("Time columns not available for delay analysis.")

,Order No,Outlet,Reason,Order_DT,Void_DT,Time Gap
2434,MR7887,Wattala,nan,2026-01-31 20:08:21,2026-02-01 09:55:09,13:46:48
1244,W71139,Kottawa,Customer denied the order,2026-01-03 11:49:00,2026-01-04 00:09:34,12:20:34
576,MK6739,Godagama,customer not respond,2026-01-21 10:55:58,2026-01-21 23:03:35,12:07:37
1212,ME7985,Kotikawatte,"Wrong location, customer nearest outlet is panadura",2026-01-11 13:23:11,2026-01-11 23:31:38,10:08:27
180,N41533,Bandarawela,Customer wanted to change the order,2026-01-30 13:23:00,2026-01-30 23:14:16,09:51:16
1761,Z90095,Negambo,Customer denied the order,2026-01-30 13:13:00,2026-01-30 23:01:35,09:48:35
1392,U53981,Kurunegala 2,Customer denied the order,2026-01-23 13:32:00,2026-01-23 22:51:24,09:19:24
2265,MR4036,Seeduwa,customer not answer,2026-01-31 13:24:02,2026-01-31 22:33:01,09:08:59
1248,W71618,Kottawa,Customer wanted to change the order,2026-01-07 13:43:00,2026-01-07 22:42:46,08:59:46
73,LZ6709,Athurugiriya,out of stock,2026-01-03 13:16:09,2026-01-03 21:57:15,08:41:06


---
## Summary

In [ ]:
# Final summary
total_value = parent_df['Amount'].sum()

display(HTML(f'''
<div style="padding:20px; border-radius:10px; color:white;">
    <h2>Report Summary</h2>
    <table style="width:100%; color:white; font-size:16px;">
        <tr><td><b>Total Void Bills:</b></td><td>{total_voids}</td></tr>
        <tr><td><b>Total Void Value:</b></td><td>Rs. {total_value:,.2f}</td></tr>
        <tr><td><b>Top Outlet:</b></td><td>{top_outlet} ({top_outlet_count} voids)</td></tr>
        <tr><td><b>Main Reason:</b></td><td>{main_reason} ({main_reason_count} cases)</td></tr>
        <tr><td><b>Number of Outlets:</b></td><td>{parent_df['Outlet'].nunique()}</td></tr>
    </table>
</div>
'''))

Total Void Bills:,713
Total Void Value:,"Rs. 2,813,411.90"
Top Outlet:,Dehiwela (25 voids)
Main Reason:,cus. Change the order (154 cases)
Number of Outlets:,114


## Export Results

In [ ]:
# Install xlsxwriter for Excel export with embedded images
!pip install xlsxwriter -q
print("Excel export package installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 4.3 MB/s eta 0:00:00
Excel export package installed!


In [21]:
# Export comprehensive report with all tables and charts to Excel
output_file = 'Void_Bills_Report_Complete.xlsx'

import io
import base64

print("Generating charts for Excel export...")

# Use matplotlib for chart generation (more reliable than kaleido in Colab)
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

# Chart 1: Channel-wise bar chart (using matplotlib)
chart_data = channel_pivot.drop('Total', errors='ignore').drop('Total', axis=1, errors='ignore').head(15)
fig1, ax1 = plt.subplots(figsize=(12, 6))
chart_data.plot(kind='bar', ax=ax1, width=0.8)
ax1.set_title('Channel-wise Void Bills by Outlet', fontsize=14, fontweight='bold')
ax1.set_xlabel('Outlet')
ax1.set_ylabel('Number of Voids')
ax1.legend(title='Order Type', loc='upper right')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('chart_channel_wise.png', dpi=150, bbox_inches='tight')
plt.close()

# Chart 2: Pie chart - outlet values
top_outlets_chart = value_pivot['Total (Rs.)'].head(10)
fig2, ax2 = plt.subplots(figsize=(10, 8))
colors_pie = plt.cm.Blues(np.linspace(0.3, 0.9, len(top_outlets_chart)))
wedges, texts, autotexts = ax2.pie(top_outlets_chart.values, labels=top_outlets_chart.index,
                                    autopct='%1.1f%%', colors=colors_pie, pctdistance=0.75)
ax2.set_title('Total Void Bill Value (Top 10 Outlets)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('chart_outlet_pie.png', dpi=150, bbox_inches='tight')
plt.close()

# Chart 3: Bar chart for all reasons
fig3, ax3 = plt.subplots(figsize=(12, 6))
colors_bar = plt.cm.Blues(np.linspace(0.3, 0.9, len(all_reasons)))
bars = ax3.barh(all_reasons['Void Reason'], all_reasons['Number Of Void'], color=colors_bar)
ax3.set_title('Void Bills by Reason Category', fontsize=14, fontweight='bold')
ax3.set_xlabel('Number of Voids')
ax3.invert_yaxis()
plt.tight_layout()
plt.savefig('chart_reasons_bar.png', dpi=150, bbox_inches='tight')
plt.close()

# Chart 4: Order type distribution
order_type_counts = parent_df['Order Type'].value_counts()
fig4, ax4 = plt.subplots(figsize=(8, 8))
colors_order = ['#4CAF50', '#2196F3', '#FFC107', '#9C27B0', '#FF5722']
wedges, texts, autotexts = ax4.pie(order_type_counts.values, labels=order_type_counts.index,
                                    autopct='%1.1f%%', colors=colors_order[:len(order_type_counts)],
                                    wedgeprops=dict(width=0.6))
ax4.set_title('Void Bills by Order Type', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('chart_order_type.png', dpi=150, bbox_inches='tight')
plt.close()

# Chart 5: Top outlets bar chart
outlet_counts_top = outlet_counts.head(15)
fig5, ax5 = plt.subplots(figsize=(12, 6))
colors_outlets = plt.cm.Reds(np.linspace(0.3, 0.9, len(outlet_counts_top)))
ax5.bar(outlet_counts_top.index, outlet_counts_top.values, color=colors_outlets)
ax5.set_title('Top 15 Outlets by Void Count', fontsize=14, fontweight='bold')
ax5.set_xlabel('Outlet')
ax5.set_ylabel('Number of Voids')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('chart_top_outlets.png', dpi=150, bbox_inches='tight')
plt.close()

print("Charts generated successfully!")
print("\nCreating Excel report with all tables...")

# Helper function to create valid sheet name (max 31 chars, no invalid chars)
def make_sheet_name(name, max_len=31):
    # Remove invalid characters for Excel sheet names
    invalid_chars = [':', '\\', '/', '?', '*', '[', ']']
    for char in invalid_chars:
        name = name.replace(char, '')
    return name[:max_len]

# Helper function to calculate auto-fit column width
def get_col_widths(df):
    """Calculate column widths based on max content length"""
    widths = {}
    for idx, col in enumerate(df.columns):
        # Get max length of column content
        max_len = max(
            df[col].astype(str).apply(len).max(),  # Max data length
            len(str(col))  # Header length
        )
        # Add padding and cap at reasonable max
        widths[idx] = min(max_len + 2, 50)
    return widths

# Helper function to add Excel Table with filters and auto-fit columns
def add_excel_table(worksheet, df, table_name, start_row=0, start_col=0):
    """Add an Excel Table with headers, filters, and auto-fit columns"""
    if len(df) == 0:
        return

    # Calculate table range
    end_row = start_row + len(df)
    end_col = start_col + len(df.columns) - 1

    # Create column definitions for the table
    columns = [{'header': str(col)} for col in df.columns]

    # Add the table with autofilter
    worksheet.add_table(start_row, start_col, end_row, end_col, {
        'name': table_name,
        'columns': columns,
        'style': 'Table Style Medium 2',
        'autofilter': True
    })

    # Auto-fit column widths
    col_widths = get_col_widths(df)
    for col_idx, width in col_widths.items():
        worksheet.set_column(start_col + col_idx, start_col + col_idx, width)

# Create Excel with xlsxwriter for image embedding
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    workbook = writer.book

    # Formats
    header_format = workbook.add_format({'bold': True, 'bg_color': '#1976D2', 'font_color': 'white', 'border': 1})
    money_format = workbook.add_format({'num_format': '#,##0.00', 'border': 1})
    number_format = workbook.add_format({'num_format': '#,##0', 'border': 1})
    text_format = workbook.add_format({'border': 1})
    title_format = workbook.add_format({'bold': True, 'font_size': 14, 'bg_color': '#1976D2', 'font_color': 'white'})

    # ===== SHEET 1: SUMMARY DASHBOARD =====
    summary_sheet = workbook.add_worksheet('Summary Dashboard')
    summary_sheet.set_column('A:A', 30)
    summary_sheet.set_column('B:B', 35)

    summary_sheet.write('A1', 'VOID BILLS ANALYSIS REPORT', title_format)
    summary_sheet.write('A2', f'Month of {month_str}', workbook.add_format({'bold': True, 'font_size': 12}))

    summary_data = [
        ('Total Void Bills', total_voids),
        ('Total Void Value', f"Rs. {total_value:,.2f}"),
        ('Top Outlet', f"{top_outlet} ({top_outlet_count} voids)"),
        ('Main Reason', f"{main_reason} ({main_reason_count} cases)"),
        ('Number of Outlets', parent_df['Outlet'].nunique()),
        ('Order Types', order_types_str),
    ]

    for i, (metric, value) in enumerate(summary_data, start=4):
        summary_sheet.write(f'A{i}', metric, header_format)
        summary_sheet.write(f'B{i}', value, text_format)

    # Insert pie chart
    summary_sheet.insert_image('D4', 'chart_order_type.png', {'x_scale': 0.7, 'y_scale': 0.7})

    # ===== SHEET 2: ALL ORDERS (with Excel Table) =====
    parent_df.to_excel(writer, sheet_name='All Orders', index=False, startrow=0)
    orders_sheet = writer.sheets['All Orders']
    add_excel_table(orders_sheet, parent_df, 'AllOrdersTable')

    # ===== SHEET 3: TOP 10 BY AMOUNT (with Excel Table) =====
    top_amount_export = parent_df.nlargest(10, 'Amount')[available_cols].copy()
    top_amount_export.to_excel(writer, sheet_name='Top 10 by Amount', index=False, startrow=0)
    top_sheet = writer.sheets['Top 10 by Amount']
    add_excel_table(top_sheet, top_amount_export, 'Top10AmountTable')

    # ===== SHEET 4: CHANNEL-WISE BREAKDOWN (with Excel Table) =====
    channel_export = channel_pivot.reset_index()
    channel_export.to_excel(writer, sheet_name='Channel-wise Breakdown', index=False, startrow=0)
    channel_sheet = writer.sheets['Channel-wise Breakdown']
    add_excel_table(channel_sheet, channel_export, 'ChannelTable')
    channel_sheet.insert_image('H3', 'chart_channel_wise.png', {'x_scale': 0.7, 'y_scale': 0.7})

    # ===== SHEET 5: OUTLET VALUES (with Excel Table) =====
    value_export = value_pivot.reset_index()
    value_export.to_excel(writer, sheet_name='Outlet Values', index=False, startrow=0)
    outlet_sheet = writer.sheets['Outlet Values']
    add_excel_table(outlet_sheet, value_export, 'OutletValuesTable')
    outlet_sheet.insert_image('H3', 'chart_outlet_pie.png', {'x_scale': 0.7, 'y_scale': 0.7})

    # ===== SHEET 6: REASON SUMMARY (with Excel Table) =====
    all_reasons.to_excel(writer, sheet_name='Reason Summary', index=False, startrow=0)
    reason_sheet = writer.sheets['Reason Summary']
    add_excel_table(reason_sheet, all_reasons, 'ReasonSummaryTable')
    reason_sheet.insert_image('D3', 'chart_reasons_bar.png', {'x_scale': 0.7, 'y_scale': 0.7})

    # ===== SHEET 7: OUTLET ANALYSIS (with Excel Table) =====
    outlet_summary = outlet_counts.reset_index()
    outlet_summary.columns = ['Outlet', 'Void Count']
    outlet_amounts = parent_df.groupby('Outlet')['Amount'].agg(['sum', 'mean', 'max']).reset_index()
    outlet_amounts.columns = ['Outlet', 'Total Value', 'Avg Value', 'Max Value']
    outlet_full = outlet_summary.merge(outlet_amounts, on='Outlet')
    outlet_full = outlet_full.sort_values('Void Count', ascending=False)

    outlet_full.to_excel(writer, sheet_name='Outlet Analysis', index=False, startrow=0)
    outlet_analysis_sheet = writer.sheets['Outlet Analysis']
    add_excel_table(outlet_analysis_sheet, outlet_full, 'OutletAnalysisTable')
    outlet_analysis_sheet.insert_image('G3', 'chart_top_outlets.png', {'x_scale': 0.7, 'y_scale': 0.7})

    # ===== SHEET 8: TOP CATEGORY #1 ORDERS (with Excel Table) =====
    if len(top_1_orders) > 0:
        sheet_name_1 = make_sheet_name(f"Top1 - {top_1_name}")
        top_1_export = top_1_orders[avail_cols].copy()
        top_1_export.to_excel(writer, sheet_name=sheet_name_1, index=False, startrow=0)
        cat1_sheet = writer.sheets[sheet_name_1]
        add_excel_table(cat1_sheet, top_1_export, 'TopCategory1Table')

    # ===== SHEET 9: TOP CATEGORY #2 ORDERS (with Excel Table) =====
    if len(top_2_orders) > 0:
        sheet_name_2 = make_sheet_name(f"Top2 - {top_2_name}")
        top_2_export = top_2_orders[avail_cols].copy()
        top_2_export.to_excel(writer, sheet_name=sheet_name_2, index=False, startrow=0)
        cat2_sheet = writer.sheets[sheet_name_2]
        add_excel_table(cat2_sheet, top_2_export, 'TopCategory2Table')

    # ===== SHEET 10: DELAYED VOIDS (with Excel Table) =====
    if 'delayed_orders' in dir() and len(delayed_orders) > 0:
        delay_export = delayed_orders[[c for c in delay_cols if c in delayed_orders.columns]].copy()
        # Format time gap as readable hours and minutes
        if 'Time_Gap_Seconds' in delay_export.columns:
            delay_export['Time Gap'] = delay_export['Time_Gap_Seconds'].apply(
                lambda s: f"{int(s // 3600)}h {int((s % 3600) // 60)}m" if pd.notna(s) else ""
            )
            delay_export = delay_export.drop('Time_Gap_Seconds', axis=1)
        delay_export.to_excel(writer, sheet_name='Delayed Voids', index=False, startrow=0)
        delay_sheet = writer.sheets['Delayed Voids']
        add_excel_table(delay_sheet, delay_export, 'DelayedVoidsTable')

    # ===== SHEET 11: VOIDER ANALYSIS (with Excel Table) =====
    if 'Void By ' in parent_df.columns:
        voider_stats = parent_df.groupby('Void By ').agg({
            order_col: 'count',
            'Amount': ['sum', 'mean', 'max']
        }).reset_index()
        voider_stats.columns = ['Void By', 'Void Count', 'Total Value', 'Avg Value', 'Max Value']
        voider_stats = voider_stats.sort_values('Void Count', ascending=False)

        voider_stats.to_excel(writer, sheet_name='Staff Voider Analysis', index=False, startrow=0)
        voider_sheet = writer.sheets['Staff Voider Analysis']
        add_excel_table(voider_sheet, voider_stats, 'VoiderAnalysisTable')

    # ===== SHEET 12: CHARTS ONLY =====
    charts_sheet = workbook.add_worksheet('Charts Gallery')
    charts_sheet.write('A1', 'VISUAL CHARTS GALLERY', title_format)
    charts_sheet.insert_image('A3', 'chart_channel_wise.png', {'x_scale': 0.55, 'y_scale': 0.55})
    charts_sheet.insert_image('A25', 'chart_reasons_bar.png', {'x_scale': 0.55, 'y_scale': 0.55})
    charts_sheet.insert_image('K3', 'chart_outlet_pie.png', {'x_scale': 0.55, 'y_scale': 0.55})
    charts_sheet.insert_image('K25', 'chart_top_outlets.png', {'x_scale': 0.55, 'y_scale': 0.55})
    charts_sheet.insert_image('T3', 'chart_order_type.png', {'x_scale': 0.55, 'y_scale': 0.55})

print(f"\nReport exported successfully: {output_file}")
print("\nSheets included:")
print("   1. Summary Dashboard - Key metrics with pie chart")
print("   2. All Orders - Complete order list")
print("   3. Top 10 by Amount - Highest value voids")
print("   4. Channel-wise Breakdown - With bar chart")
print("   5. Outlet Values - With pie chart")
print("   6. Reason Summary - With bar chart")
print("   7. Outlet Analysis - Detailed outlet stats with chart")
print(f"   8. Top1 - {top_1_name[:24]}")
print(f"   9. Top2 - {top_2_name[:24]}")
print("  10. Delayed Voids")
print("  11. Staff Voider Analysis")
print("  12. Charts Gallery - All charts in one sheet")

# Download the file
files.download(output_file)
print(f"\nDownload started: {output_file}")

Generating charts for Excel export...
Charts generated successfully!

Creating Excel report with all tables...

Report exported successfully: Void_Bills_Report_Complete.xlsx

Sheets included:
   1. Summary Dashboard - Key metrics with pie chart
   2. All Orders - Complete order list
   3. Top 10 by Amount - Highest value voids
   4. Channel-wise Breakdown - With bar chart
   5. Outlet Values - With pie chart
   6. Reason Summary - With bar chart
   7. Outlet Analysis - Detailed outlet stats with chart
   8. Top1 - cus. Change the order
   9. Top2 - promotion
  10. Delayed Voids
  11. Staff Voider Analysis
  12. Charts Gallery - All charts in one sheet


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Download started: Void_Bills_Report_Complete.xlsx


---
### Report Complete!